# Partitioning and Glue

### Introduction

In the last lesson, we saw how we can use awswrangler to read and write to files.

```python
# read from object url
crimes_df = wr.s3.read_parquet(crimes_parquet_url)

# write to object specifying df, and url
wr.s3.to_parquet(df=s3_df, path=write_url)
```

And that we can read and write to folders with the `dataset = True` argument.

In this lesson, we'll see how we can better organize and query the data stored in s3 by using partitioning, glue, and athena.

### Partitioning

Now when writing to a folder, it's often a good idea to partition our data.  This way, if we want to query our s3 dataset, tools like athena or spark will not search through *all* of the files in the dataset, but just those in the matching partition.

Let's begin by loading our data into a dataframe -- it should be in your bucket, or you can read it from the data directory. 

In [2]:
import awswrangler as wr
bucket_name = "jigsaw-labs-student" # change bucket name
folder_name = "chicago"

crimes_df = wr.s3.read_parquet(path=f"s3://{bucket_name}/{folder_name}/",
                dataset=True)

Now when partitioning, we generally want to choose a column that we would often filter by.  For example, with our chicago crimes dataset, date would be a good parameter, as then could filter by date.  Or perhaps we should choose the kind of crime, or the neighborhood.  

For this lesson, we'll partition by year.  We can see that there are three different years in our dataset.

In [3]:
crimes_df.Year.unique()

['2012.0', '2015.0', '2016.0']
Categories (3, object): ['2012.0', '2015.0', '2016.0']

Now let's store our dataset, partitioning by year.

We do so by adding a `partition_cols` argument, specifying the column or columns that we want to paritition by.

In [55]:
wr.s3.to_parquet(df=crimes_df, 
                path=f"s3://{bucket_name}/{folder_name}/",
                partition_cols = ['Year'],
                dataset=True)

/Users/jeffreykatz/opt/anaconda3/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:150: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


{'paths': ['s3://jigsaw-labs-student/chicago/Year=2012.0/6ca60edb4e1b491c8af4302ce6b69423.snappy.parquet',
  's3://jigsaw-labs-student/chicago/Year=2015.0/6ca60edb4e1b491c8af4302ce6b69423.snappy.parquet',
  's3://jigsaw-labs-student/chicago/Year=2016.0/6ca60edb4e1b491c8af4302ce6b69423.snappy.parquet'],
 'partitions_values': {'s3://jigsaw-labs-student/chicago/Year=2012.0/': ['2012.0'],
  's3://jigsaw-labs-student/chicago/Year=2015.0/': ['2015.0'],
  's3://jigsaw-labs-student/chicago/Year=2016.0/': ['2016.0']}}

This time, if we look at the bucket, we'll that our data was partitioned into a separate folder per year. 

<img src="./partitioned.png" width="70%">

### Connecting to glue

Ok, so now that we've partitioned our data, we can allow a database like Athena or Spark to read from our S3 bucket.  Here, we'll use Athena, which means we first need Glue to scan the relevant s3 folder.

Let's start by getting a list of databases in Glue's catalog.

In [57]:
databases = wr.catalog.databases()
# databases

And now let's create a new database called `chicago_datasets`.

In [58]:
wr.catalog.create_database("chicago_datasets")

If we look at databases again, we'll see it listed there.

In [ ]:
databases = wr.catalog.databases()
databases

And finally, we'll get glue to scan our dataset by using the `store_parquet_metadata` function.  

Notice that we specify the `database`, and the table name (`crimes`) that we want the dataset referenced as.

In [59]:
bucket_name = "jigsaw-labs-student" # change bucket name
folder_name = "chicago"

path = f"s3://{bucket_name}/{folder_name}/"

res = wr.s3.store_parquet_metadata(
    path=path,
    database="chicago_datasets",
    table="crimes",
    dataset=True,
    mode="overwrite"
)

Let's see what it came up with -- if we specify the table name we can see the columns and datatypes of the crime table.

In [62]:
wr.catalog.table(database="chicago_datasets", table="crimes").T[:2]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Column Name,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,updated_on,latitude,longitude,location,year
Type,bigint,string,string,string,string,string,string,string,boolean,boolean,...,double,double,string,double,double,string,double,double,string,string


So it has found various columns and the related datatypes.  Finally, let's use athena to query our created table.

In [69]:
query = "SELECT * FROM crimes where Year > '2015' limit 10"
crimes_2015_df = wr.athena.read_sql_query(query, 
                                        database="chicago_datasets")

In [70]:
crimes_2015_df[:2]

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,updated_on,latitude,longitude,location,year
0,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,...,24.0,29.0,08B,1154907.0,1893681.0,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)",2016.0
1,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,20.0,42.0,08B,1183066.0,1864330.0,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)",2016.0


> **One last thing**: Notice that we had to query the year by the string, `'2015'`.  For it to have been an integer, we should have coerced the column to an integer in pandas, and then stored it as a parquet file, and scanned with Glue.

### Summary

In this lesson, we saw how to both partition and store metadata about our dataset.

We partitioned our data using the `to_parquet` method's `partition_cols` arguments.  And we chose to partition by the year column.

In [ ]:
wr.s3.to_parquet(df=crimes_df, 
                path=f"s3://{bucket_name}/{folder_name}/",
                partition_cols = ['Year'],
                dataset=True)

From there, we worked with the Glue catalog to create a database and store information about our relevant s3 folder so it can be queried by athena.

```python
# create a db
wr.catalog.create_database("chicago_datasets")

# store metadata about the dataset creating a new table
res = wr.s3.store_parquet_metadata(
    path=path,
    database="chicago_datasets",
    table="crimes",
    dataset=True,
    mode="overwrite"
)

# view resulting schema
wr.catalog.table(database="chicago_datasets", table="crimes")
```

And finally we queried our bucket using athena.

```python
query = "SELECT * FROM crimes where Year > '2015' limit 10"
crimes_2015_df = wr.athena.read_sql_query(query, 
                                        database="chicago_datasets")
```

### Resources

[AWS Wrangler Tutorial](https://github.com/aws/aws-sdk-pandas/blob/main/tutorials/004%20-%20Parquet%20Datasets.ipynb)

[Crawling in Glue](https://github.com/aws/aws-sdk-pandas/blob/main/tutorials/010%20-%20Parquet%20Crawler.ipynb)

[Glue Partitioning](https://docs.aws.amazon.com/athena/latest/ug/partition-projection.html)